In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import gradio as gr
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import warnings

In [3]:
warnings.filterwarnings('ignore')
# from PIL import Image
print('Gradio version: ', gr.__version__)

Gradio version:  3.35.2


In [4]:
# define dataset
dataset = pd.read_csv('D:\projects-Georgian\SmallPOX\SyMPox\Data.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [5]:
# Oversample with SMOTE and random undersample for imbalanced dataset
# define pipeline
over = SMOTE(sampling_strategy=0.5, random_state=27) #8
under = RandomUnderSampler(sampling_strategy=0.5, random_state=376) #1
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

In [6]:

# transform the dataset
X, y = pipeline.fit_resample(X, y)

# split dataset to train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=354)

In [7]:
# define model and make prediction on test set
classifier = XGBClassifier(learning_rate=0.0991, gamma=0, n_estimators = 80)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [8]:
# confusion matrix & accuracy score
cm = confusion_matrix(y_test, y_pred)    
print('\n', cm)
print('\naccuracy:   ', accuracy_score(y_test, y_pred))


 [[15  2]
 [ 1 38]]

accuracy:    0.9464285714285714


In [9]:
# creating a graphic environment
header = dataset.columns[1:-1]

def predict(symptoms):
    new_sample = [0]*len(header)
    for i in range(len(header)):
        for j in range(len(symptoms)):
            if header[i] == symptoms[j]:
                new_sample[i] = 1
    pred = classifier.predict(np.array(new_sample).reshape(1, len(header)))
    print(pred)
    
    if pred == 1:
      return  'positive.png', '❌ Regrettably, there exists a possibility of Monkeypox.'
      
    else:
      return  'negative.png', '✔️ Fortunately, you have been spared from the presence of Monkeypox. It is recommended to consult with a medical professional for further confirmation.'
    
    
title='🐒🦠 Detection of Monkeypox Cases Based on Symptoms'
des='1. Kindly choose the symptoms you are experiencing from the provided section below and proceed by clicking the <b>Submit</b> button. The outcome will be displayed in the <u>output</u> section for your reference\n2. If you wish to reset the selected symptoms, simply click the <b>Clear</b> button.'

out_text = gr.Textbox(label="Result")
out_image = gr.Image(label=" ")
demo = gr.Interface(fn=predict, inputs=gr.CheckboxGroup(list(header)), outputs=[out_image, out_text], allow_flagging="never",
                   title=title, description=des)

if __name__ == "__main__":
    demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [31]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import plotly_express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
%config InlineBackend.figure_format ='retina'

In [30]:
pip install wordcloud

  Using cached wordcloud-1.9.3-cp310-cp310-win_amd64.whl (299 kB)
Note: you may need to restart the kernel to use updated packages.


In [33]:
Case_worldwide= pd.read_csv('D:\projects-Georgian\SmallPOX\Monkey_Pox_Cases_Worldwide.csv')

In [34]:
Case_worldwide.head()

,Country,Confirmed_Cases,Suspected_Cases,Hospitalized,Travel_History_Yes,Travel_History_No
0,England,3412.0,0.0,5.0,2.0,7.0
1,Portugal,908.0,0.0,0.0,0.0,34.0
2,Spain,7083.0,0.0,13.0,2.0,0.0
3,United States,24403.0,0.0,4.0,41.0,11.0
4,Canada,1388.0,12.0,1.0,5.0,0.0


In [35]:
Case_worldwide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             129 non-null    object 
 1   Confirmed_Cases     129 non-null    float64
 2   Suspected_Cases     129 non-null    float64
 3   Hospitalized        129 non-null    float64
 4   Travel_History_Yes  129 non-null    float64
 5   Travel_History_No   129 non-null    float64
dtypes: float64(5), object(1)
memory usage: 6.2+ KB


In [36]:
# Monkey Pox affected countries worldwide

def plot_map(df,col, title):
    df = df[df[col]>0]
    fig = px.choropleth(df, locations="Country", locationmode='country names', 
                  color=col, hover_name="Country", 
                  title=title, hover_data=[col], color_continuous_scale="Plasma")
    fig.show()

In [37]:
def treemap_plot(col):
    fig = px.treemap(mp_cases_worldwide, path=["Country"], values=col, height=700,
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.data[0].textinfo = 'label+text+value'
    fig.show()

In [38]:
plot_map(Case_worldwide,'Confirmed_Cases', 'Confirmed Cases by Country')